# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [196]:
# Import libraries
import numpy as np
import pandas as pd

In [197]:
# Importing the dataset

pokemon = pd.read_csv('pokemon.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [198]:
# Testing my hypothesis: 

pokemon_numeric = pokemon.select_dtypes(np.number).drop(['Generation', '#'], axis = 1)
pokemon_numeric.head()

# 1. I check for correlation between the elements: 

for elem in list(pokemon_numeric.columns):
    print ('The correlation of ', elem, ' with Total is: ', pokemon_numeric['Total'].corr(pokemon_numeric[elem]))
    
    #RESULT: Correlation values do not seem very high! 
    
    
# 2. Is there a formular to compute Total based on the elements? 

list_sum = pokemon_numeric.drop(['Total'], axis = 1).columns.to_list()
print(list_sum)  

pokemon_numeric['sum_of_elem'] = pokemon_numeric[list_sum].sum(axis=1)
display(pokemon_numeric.head())

    #RESULT: on first sight it seems that the total is calculated by the SUM of the elements
    

# 3. Are their exceptions for pokemons?:

display(pokemon_numeric[pokemon_numeric['Total'] != pokemon_numeric['sum_of_elem']])

    # RESULT: there seems to be no exception

# 4. I want to also check for Null Values: 

pokemon_numeric.isna().sum()



The correlation of  Total  with Total is:  1.0
The correlation of  HP  with Total is:  0.6187483543135468
The correlation of  Attack  with Total is:  0.7362106519949297
The correlation of  Defense  with Total is:  0.6127874293532546
The correlation of  Sp. Atk  with Total is:  0.7472498608443285
The correlation of  Sp. Def  with Total is:  0.7176094700519234
The correlation of  Speed  with Total is:  0.5759426553220428
['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']


,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,sum_of_elem
0,318,45,49,49,65,65,45,318
1,405,60,62,63,80,80,60,405
2,525,80,82,83,100,100,80,525
3,625,80,100,123,122,120,80,625
4,309,39,52,43,60,50,65,309


,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,sum_of_elem


Total          0
HP             0
Attack         0
Defense        0
Sp. Atk        0
Sp. Def        0
Speed          0
sum_of_elem    0
dtype: int64

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [216]:
from sklearn.preprocessing import OneHotEncoder

!pip show scikit-learn 

Name: scikit-learn
Version: 1.1.1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /Users/steib/opt/anaconda3/lib/python3.9/site-packages
Requires: scipy, numpy, joblib, threadpoolctl
Required-by: scikit-learn-intelex


In [217]:
# 1. OneHotEncoder and fit_transform()


enc = OneHotEncoder(sparse = True)
feature_array = enc.fit_transform(pokemon[['Type 1', 'Type 2']]).toarray()
encoded_types = pd.DataFrame(feature_array)


# 2. extracting the input column names of the 2 array feature: 

            # !enc.get_feature_names_out()! 
            # AttributeError: 'OneHotEncoder' object has no attribute 'get_feature_names_out'

col = enc.categories_ # a list of 2 arrays

def rename1(x):
    return ('T1_'+x)

def rename2(x):
    x = str(x)
    return ('T2_'+x)

col[0] = list(map(rename1, col[0]))
col[1] = list(map(rename2, col[1]))    
    
col_labels = np.concatenate((col[0], col[1]), axis = 0)
col_labels

encoded_types.columns = col_labels

encoded_types




,T1_Bug,T1_Dark,T1_Dragon,T1_Electric,T1_Fairy,T1_Fighting,T1_Fire,T1_Flying,T1_Ghost,T1_Grass,...,T2_Grass,T2_Ground,T2_Ice,T2_Normal,T2_Poison,T2_Psychic,T2_Rock,T2_Steel,T2_Water,T2_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [259]:
# your code here

encoded_types['Total'] = pokemon['Total']
encoded_types.head()


# 1. Checking for correlation


for elem in list(encoded_types.columns):
    if encoded_types['Total'].corr(encoded_types[elem]) > 0.1:
        print ('\nThe correlation of ', elem, ' with Total is: ', encoded_types['Total'].corr(encoded_types[elem]))
    
    #RESULT: Based on the correlation with the encoded values I would chose a POKEMON from the TYPE 1 DRAGON


The correlation of  T1_Dragon  with Total is:  0.196531656074697

The correlation of  T2_Dragon  with Total is:  0.11524034126366932

The correlation of  T2_Fighting  with Total is:  0.13872559558437536

The correlation of  T2_Ice  with Total is:  0.10086989669829381

The correlation of  Total  with Total is:  1.0


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [258]:
# My approach: 

# 1. Combine the Type 1 and Type 2 to a Combined Power Value
# 2. fit_transform OneHotEncoder
# 3. Caculate the correlation between Combined Power Value and Total
# 4. Choose the Combination with the highest positiv correlation


## ad 1: 

def combo(row):
    if pd.isnull(row[1]):
        return row[0] 
    else:  
        return (row[0] + '-' + row[1])
    

CPV = pokemon[['Type 1', 'Type 2']].apply(tuple, axis = 1) 
pokemon['CPV'] = pd.DataFrame(list(map(combo, CPV)))


## ad 2:

enc = OneHotEncoder()
CPV = pd.DataFrame(enc.fit_transform(pokemon[['Combo Type']]).toarray())
col = enc.get_feature_names()

CPV.columns = col

## ad 3: 

CPV['Total'] = pokemon['Total']

for elem in list(CPV.columns):
    if CPV['Total'].corr(CPV[elem]) > 0.1:
        print ('\nThe correlation of ', elem, ' with Total is: ', CPV['Total'].corr(CPV[elem]))
    
    
    
    # RESULT: I would choose TYPE 1: DRAGON , TYPE 2: FLYING


The correlation of  x0_Dragon-Flying  with Total is:  0.14977678401772032

The correlation of  x0_Dragon-Ice  with Total is:  0.12873729749187376

The correlation of  x0_Dragon-Psychic  with Total is:  0.12706588538942937

The correlation of  x0_Ghost-Dragon  with Total is:  0.10226382809208294

The correlation of  x0_Psychic-Fighting  with Total is:  0.10417342433978881

The correlation of  Total  with Total is:  1.0
